In [1]:
#Assignment:

#1) Anon the user field and create a mapping
#2) Anon the IP address field (which is expressed as a string, but can also map to a 32bit integer. [yet sparsely so in our data]
#3) Make a vectors of the anonymous data and clear data
#4) Generate a "ragged array" of the data in this form (i.e. dump out the records which are of variable length)
### Bonus) Tell us which users jobs fail and how often as an absolute number, and as a fraction of the whole dataset.

### First pass of exploring FNAL data

In [2]:
import sys, os
import pandas as pd
import glob

In [3]:
DATA_DIR = "../data"

In [4]:
lib_path = '../'
if lib_path not in sys.path:
    sys.path.append(lib_path)

In [5]:
#!pip install fastparquet

In [6]:
fname_in = f"{DATA_DIR}/fifebatch-history-v3-2024.01.01-0002.parquet"
fname_out = f"{DATA_DIR}/anonymized-batch-0002.parquet"

In [7]:
fname_hmac_out = f"{DATA_DIR}/hmac-anonymized-batch-0002.parquet"
fname_fpe_out = f"{DATA_DIR}/fpe-anonymized-batch-0002.parquet"

In [8]:
fnames_in = f"{DATA_DIR}/fifebatch-history-*.parquet"
files = glob.glob(fnames_in)
print(f"Found {len(files)} files")
# Read and concatenate
df_all = pd.concat([pd.read_parquet(f, engine="fastparquet") for f in files], ignore_index=True)

Found 6 files


In [9]:
df = pd.read_parquet(fname_in, engine="fastparquet")  # or engine="fastparquet"
print(df.dtypes)       # Column names + types

@timestamp                datetime64[us]
@version                          object
AccountingGroup                   object
AccountingGroupOSG                object
AcctGroup                         object
                               ...      
x509UserProxyFirstFQAN            object
x509UserProxyVOName               object
x509userproxy                     object
x509userproxysubject              object
xcount                            object
Length: 595, dtype: object


In [10]:
[col for col in df.columns.tolist() if 'fail' in col.lower()]

['DAG_NodesFailed']

In [11]:
df.head()

,@timestamp,@version,AccountingGroup,AccountingGroupOSG,AcctGroup,AcctGroupUser,AllowOpportunistic,Args,Arguments,AutoClusterAttrs,...,use_x509userproxy,x509UserProxyEmail,x509UserProxyExpiration,x509UserProxyExpiration_ms,x509UserProxyFQAN,x509UserProxyFirstFQAN,x509UserProxyVOName,x509userproxy,x509userproxysubject,xcount
0,NaT,1,group_gm2.prod.gm2pro,None,None,None,None,multipleroot True muon True noGrid False input...,None,None,...,None,None,2024-01-07 16:58:02,1.704668e+12,None,None,None,gm2pro.Production.proxy,/DC=org/DC=incommon/C=US/ST=Illinois/O=Fermi R...,None
1,NaT,1,group_icarus.pro.icaruspro,None,None,None,None,--debug --find_setups --source-unquote /cvmfs/...,None,None,...,None,None,2024-01-07 16:58:07,1.704668e+12,None,None,None,icaruspro.Production.proxy,/DC=org/DC=incommon/C=US/ST=Illinois/O=Fermi R...,None
2,NaT,1,group_icarus.pro.icaruspro,None,None,None,None,--debug --find_setups --source-unquote /cvmfs/...,None,None,...,None,None,2024-01-07 16:58:07,1.704668e+12,None,None,None,icaruspro.Production.proxy,/DC=org/DC=incommon/C=US/ST=Illinois/O=Fermi R...,None
3,NaT,1,group_uboone.prod.uboonepro,None,None,None,None,--nfile 1 --group uboone -g -c wrapper.fcl --u...,None,None,...,None,None,2024-01-07 16:58:08,1.704668e+12,None,None,None,uboonepro.Production.proxy,/DC=org/DC=incommon/C=US/ST=Illinois/O=Fermi R...,None
4,NaT,1,group_pip2.mars.amakovec,None,None,None,None,None,None,None,...,None,amakovec@fnal.gov,2024-01-07 15:29:15,1.704663e+12,None,None,None,x509up_pip2_MARS_59791,/DC=org/DC=cilogon/C=US/O=Fermi National Accel...,None


In [12]:
from hashlib import sha256
from lib.anonymize_kv_mvp import anonymize_df
from lib.utils_secrets import gen_secret_hex

In [13]:
anonymized_columns = ["x509UserProxyEmail","User","JobsubClientIpAddress"]
#anonymized_columns = ["AccountingGroup","x509UserProxyEmail","x509userproxy","x509userproxysubject","JobsubClientIpAddress"],

In [14]:
df[anonymized_columns + ['DAG_NodesFailed']]

,x509UserProxyEmail,User,JobsubClientIpAddress,DAG_NodesFailed
0,None,gm2pro@fnal.gov,131.225.164.43,NaN
1,None,icaruspro@fnal.gov,131.225.240.90,NaN
2,None,icaruspro@fnal.gov,131.225.240.90,NaN
3,None,uboonepro@fnal.gov,131.225.240.146,NaN
4,amakovec@fnal.gov,amakovec@fnal.gov,131.225.240.190,NaN
...,...,...,...,...
37495,None,icaruspro@fnal.gov,131.225.240.90,NaN
37496,None,icaruspro@fnal.gov,131.225.240.90,NaN
37497,None,uboonepro@fnal.gov,131.225.240.146,NaN
37498,None,icaruspro@fnal.gov,131.225.240.90,NaN


In [22]:
df[~df['DAG_NodesFailed'].isna()]

,@timestamp,@version,AccountingGroup,AccountingGroupOSG,AcctGroup,AcctGroupUser,AllowOpportunistic,Args,Arguments,AutoClusterAttrs,...,use_x509userproxy,x509UserProxyEmail,x509UserProxyExpiration,x509UserProxyExpiration_ms,x509UserProxyFQAN,x509UserProxyFirstFQAN,x509UserProxyVOName,x509userproxy,x509userproxysubject,xcount


In [15]:
#key_hex = sha256(b"kms-secret").hexdigest()
key_hex = gen_secret_hex(mode="dev", data_dir="../data")
df_hmac, mapping = anonymize_df(
    df,
    columns=anonymized_columns,
    key_hex=key_hex,
    mode="hmac",
    email_preserve_domain=True,   # token@domain
)

In [16]:
df_hmac[anonymized_columns].head(5)

,x509UserProxyEmail,User,JobsubClientIpAddress
0,None,sdgawgbkhw7ld4ig3ndebc,iqkl5uv7w73225x7j2q7nk
1,None,gakaqchwlk37xxojwxepxg,7ebx4f2vusi5w463zilxcz
2,None,gakaqchwlk37xxojwxepxg,7ebx4f2vusi5w463zilxcz
3,None,4qvket2bv3jfhsqcxavpcl,zxjln6il7ptfcjxoahf5oe
4,g37s3mev57zmgqxb@fnal.gov,vh3x2wljnt6yrtymwggnzr,wpiztgq6ffvjw2ezwadub6


In [17]:
df_hmac.to_parquet(fname_hmac_out, index=False, engine="fastparquet")

In [18]:
df_fpe, mapping = anonymize_df(
    df,
    #columns=["AccountingGroup","x509UserProxyEmail","x509userproxy","x509userproxysubject"],
    columns=["x509userproxy"],
    key_hex=key_hex,
    mode="fpe",
    email_preserve_domain=True,   # only local-part anonymized
)

In [19]:
df_fpe.to_parquet(fname_fpe_out, index=False, engine="fastparquet")

In [20]:
len(mapping)

37500